In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def create_faster_rcnn_model(num_classes):
    # Backbone CNN (e.g., ResNet50 as feature extractor)
    backbone = keras.applications.ResNet50(
        include_top=False, weights='imagenet', input_shape=(224, 224, 3)
    )
    
    # Region Proposal Network (RPN)
    rpn = keras.models.Sequential([
        layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
        layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
        layers.Conv2D(512, (1, 1), activation='relu'),
        layers.Conv2D(18, (1, 1))  # 9 anchors with 2 coordinates each (x, y, w, h)
    ])
    
    # Region-based CNN (RCNN) for classification and bounding box regression
    rcnn_head = keras.models.Sequential([
        layers.Flatten(),
        layers.Dense(1024, activation='relu'),
        layers.Dense(1024, activation='relu')
    ])
    
    classification_head = layers.Dense(num_classes, activation='softmax', name='classification')
    bbox_regression_head = layers.Dense(4, activation='linear', name='bounding_box')
    
    # Inputs
    input_image = layers.Input(shape=(224, 224, 3), name='image')
    proposals = layers.Input(shape=(None, 4), name='proposals')
    
    # Backbone CNN
    features = backbone(input_image)
    
    # RPN to generate region proposals
    rpn_output = rpn(features)
    
    # RCNN
    rcnn_input = layers.RoiAlign((7, 7))([features, proposals])
    rcnn_output = rcnn_head(rcnn_input)
    classification_output = classification_head(rcnn_output)
    bbox_output = bbox_regression_head(rcnn_output)
    
    # Model
    model = keras.Model(inputs=[input_image, proposals], outputs=[classification_output, bbox_output])
    
    return model

# Create the Faster R-CNN model
num_classes = len(objects)
faster_rcnn_model = create_faster_rcnn_model(num_classes)

# Compile the model
faster_rcnn_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss={
        'classification': 'sparse_categorical_crossentropy',
        'bounding_box': 'mse'
    },
    metrics={'classification': 'accuracy'}
)

# Print model summary
faster_rcnn_model.summary()
